# Vamos a sintetizar un conjunto de datos a partir del extract de spotify

In [1]:
from pathlib import Path 
import json

import pandas as pd
import altair as alt
import sidetable
import janitor
from rich import print 


In [2]:
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [3]:
mydata = Path(".").joinpath("MyData")

In [4]:
history = []
for file in mydata.glob("Streaming*.json"):
    with open(file, "r") as json_file:
        history.extend(json.load(json_file))

In [5]:
print(history[0])

{'endTime': '2021-12-28 00:00', 'artistName': 'Desert Niños', 'trackName': 'American Party', 'msPlayed': 274100}

In [6]:
data = pd.DataFrame(history).clean_names().rename(columns= {"artistname": "artist", "trackname": "track"})

In [7]:
data.shape

(10232, 4)

In [8]:
data.head()

,endtime,artist,track,msplayed
0,2021-12-28 00:00,Desert Niños,American Party,274100
1,2021-12-28 00:05,Desert Niños,Luz de Luna,279300
2,2021-12-28 00:09,Desert Niños,Kayak,245000
3,2021-12-28 00:13,Desert Niños,Calidad,201200
4,2021-12-28 00:16,Desert Niños,Lobos,197600


In [9]:
data.groupby(['artist', 'track']).count()

endtime  msplayed
artist                               track                                  
A Tribe Called Quest                 Excursions                  1         1
A$AP Ferg                            Plain Jane                  2         2
A.C.O                                De Madrugada                8         8
                                     El encendedor               1         1
Ab-Soul                              D.R.U.G.S.                  4         4
...                                                            ...       ...
sofia shizuko                        chichiriviche               7         7
Ídolo: The Ballad of Chalino Sánchez 1. Death Note               2         2
                                     1. La nota de muerte        1         1
Ñengo El Quetzal                     El Rikis                    1         1
                                     Plakka Plakka               1         1

[1928 rows x 2 columns]

In [10]:
data.stb.freq(['artist', 'track'])

,artist,track,count,percent,cumulative_count,cumulative_percent
0,La Plebada,Placoseo Como Se Debe,139,1.358483,139,1.358483
1,La Plebada,El R8,129,1.260751,268,2.619234
2,La Plebada,La Galliza,124,1.211884,392,3.831118
3,La Banda Baston,Lupe Esparza,110,1.075059,502,4.906177
4,La Plebada,Del Año,106,1.035966,608,5.942142
...,...,...,...,...,...,...
1923,Akapellah,Como Mario,1,0.009773,10228,99.960907
1924,Akapellah,Cielo Gris,1,0.009773,10229,99.970680
1925,Adn Maya Colectivo,Laayli' kuxa'ano'one,1,0.009773,10230,99.980453
1926,A.C.O,El encendedor,1,0.009773,10231,99.990227


In [11]:
data.stb.freq(['artist'])

,artist,count,percent,cumulative_count,cumulative_percent
0,La Plebada,1243,12.148163,1243,12.148163
1,La Banda Baston,1096,10.711493,2339,22.859656
2,Fntxy,601,5.873729,2940,28.733385
3,Desert Niños,405,3.958170,3345,32.691556
4,Yoga Fire,389,3.801798,3734,36.493354
...,...,...,...,...,...
475,Althea And Donna,1,0.009773,10228,99.960907
476,Alejandro Fernández,1,0.009773,10229,99.970680
477,Al2 El Aldeano,1,0.009773,10230,99.980453
478,Adn Maya Colectivo,1,0.009773,10231,99.990227


In [12]:
data.stb.missing()

,missing,total,percent
endtime,0,10232,0.0
artist,0,10232,0.0
track,0,10232,0.0
msplayed,0,10232,0.0


In [13]:
podcast_names = [
    "NO SKIPS with Jinx and Shea",
    "Just Break Up",
    "Normal Gossip",
    "Questlove Supreme",
    "El Flowcast", 
    "RADIOSUENARECIO!",
    "The Sum of Us",
    "Ídolo: The Ballad of Chalino Sánchez",
    "The Cipher",
    "La Cima",
    "Chats with Cat",
    "Dissect", 
    "The Latin Alternative",
    "LOUD",
    "Draws in Spanish |  Conversations with Latinx Visual Artists and Designers",
    "Mano a Mano",
    "Entiende Tu Mente",
    "Lo Que Ando Oyendo",
    "LobaLand: Wellness Experience",
    "California Love",
    "How to Save a Planet",
    "WILD",
]

In [14]:
mask_msplayed = data['msplayed'] >= 30_000 # 30 second
mask_podcasts = data['artist'].str.strip().isin(podcast_names)

In [15]:
data[mask_msplayed & ~mask_podcasts]['artist'].value_counts().tail(110)

Selena                   1
Kase.O                   1
Lupillo Rivera           1
Kid Dee                  1
Mime 871                 1
                        ..
Los Bunkers              1
Los Amigos Invisibles    1
Mœnia                    1
Flor De Rap              1
ROSE BEAT                1
Name: artist, Length: 110, dtype: int64

In [16]:
podcast_data = data[mask_podcasts].copy()
data = data[mask_msplayed & ~mask_podcasts]

In [17]:
podcast_data

,endtime,artist,track,msplayed
122,2021-12-29 16:54,El Flowcast,Adán Cruz | Episodio 3,550591
473,2022-01-07 03:12,El Flowcast,Eptos Uno | Episodio 13,3441293
474,2022-01-07 03:12,El Flowcast,Miky Woodz | Episodio 14,32771
487,2022-01-07 14:57,California Love,Prologue,433516
546,2022-01-10 01:00,NO SKIPS with Jinx and Shea,Mos Def: ‘Black on Both Sides’ | Vol. 2 Ep. 10,793356
...,...,...,...,...
9155,2022-11-01 03:45,The Sum of Us,"Lewiston, ME: A New Crop",530705
9703,2022-11-25 13:08,Entiende Tu Mente,"Ser ""demasiado"" racional | 225",30160
9704,2022-11-25 13:19,Entiende Tu Mente,"Ser ""demasiado"" racional | 225",530840
9767,2022-11-26 16:06,Entiende Tu Mente,"Ser ""demasiado"" racional | 225",731075


In [18]:
data.sort_values(by="msplayed", ascending = False).tail(50)

,endtime,artist,track,msplayed
2675,2022-03-26 22:49,La Plebada,Por la Mala,32937
4407,2022-06-02 01:37,La Banda Baston,Roberto Gómez Bolaños,32883
1570,2022-02-21 03:56,Mantequilloso,Espera.,32861
4033,2022-05-16 19:57,Bad Bunny,Yo No Soy Celoso,32853
4317,2022-05-30 03:29,Wk,Todo Está Bien,32848
2565,2022-03-25 03:05,La Plebada,Por la Mala,32798
1897,2022-03-05 17:48,Aleman,Cabrón,32684
5528,2022-07-15 14:54,Daddy Yankee,Mix Rap 1: 30-30 / Mi Fanatico / Se Acelera el...,32566
2983,2022-04-05 18:57,La Plebada,Bien Jalado,32504
1037,2022-01-31 15:58,Molotov,Más Vale Cholo - Desde El Palacio De Los Deportes,32391


In [19]:
alt.Chart(data).mark_bar().encode(
    x = alt.X("msplayed", bin = alt.Bin(maxbins=50)),
    y = alt.Y("count()"),
)

/Users/sergio/projects/chekosdev-sandbox/.venv/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

In [20]:

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from dotenv import dotenv_values

config = dotenv_values("../.env")

client_id = config.get("SPOTIPY_CLIENT_ID", " ")
client_secret = config.get("SPOTIPY_CLIENT_SECRET", " ")

sp = spotipy.Spotify(
    client_credentials_manager=SpotifyClientCredentials(
        client_id=client_id, client_secret=client_secret
    )
)


In [21]:
def get_track_info(query: str, is_uri: bool = False):
    if not is_uri:
        search_result = sp.search(query, limit=1, type="track")
        if search_result["tracks"]["items"]:
            track_item = search_result["tracks"]["items"][0]
        else:
            return False
    else:
        track_item = sp.track(query)

## Check if already in uniqueTracks

In [22]:
with open("minMetadataTracks.json", "r") as file:
    unique_tracks = json.load(file)

In [23]:
already_saved = pd.DataFrame(unique_tracks)
already_saved['saved'] = True
already_saved

,artist,album,track,uri,saved
0,Mike Diaz,Renace,Kintsugi,spotify:track:00TPikzhlAO79taN4K8VDK,True
1,Mexican Institute Of Sound,Méjico Máxico,Mirando a las muchachas,spotify:track:00Z3s0Xw7oE3fgTSZVjEFs,True
2,Cartel De Santa,Viejo Marihuano,Leve,spotify:track:00jpfco62YfQj1BBYCCuKH,True
3,El Perro Gamboa,Núm3Ros,Núm3Ros,spotify:track:00qVP7ZX0UJqvqJrhZWgYq,True
4,La Banda Baston,Degeneración Nacional,Cuando,spotify:track:00r6SXWE6lvfikdbsl6qSR,True
...,...,...,...,...,...
903,Santa Fe Klan,Ojos Tumbados,Ojos Tumbados,spotify:track:7uYFOLEhhmojSLTyVZ8r4O,True
904,José José,Reencuentro,El Amar y el Querer,spotify:track:7wUt1v2ddVqPayC8EKDp5P,True
905,Gera MX,Se Me Olvidó,Se Me Olvidó,spotify:track:7xLYLM5K6S1TwiSdfuhZQg,True
906,Bizarrap,"Aleman: Bzrp Music Sessions, Vol. 15","Aleman: Bzrp Music Sessions, Vol. 15",spotify:track:7xRWmHZ2wt3zDgM5mAL0JU,True


In [24]:
data.set_index(['artist', 'track']).join(already_saved.set_index(["artist", "track"]))

endtime  msplayed  \
artist               track                                       
A Tribe Called Quest Excursions     2022-01-28 21:59     41865   
A$AP Ferg            Plain Jane     2022-03-27 02:20    173600   
                     Plain Jane     2022-11-29 01:31    173600   
A.C.O                De Madrugada   2022-01-21 02:55    220757   
                     De Madrugada   2022-07-18 15:48    220757   
...                                              ...       ...   
sofia shizuko        chichiriviche  2022-07-15 00:49    149000   
                     chichiriviche  2022-07-15 00:55    149000   
                     chichiriviche  2022-07-16 18:59     69525   
                     chichiriviche  2022-08-15 19:58    149000   
Ñengo El Quetzal     Plakka Plakka  2022-07-20 18:45    181054   

                                             album  \
artist               track                           
A Tribe Called Quest Excursions                NaN   
A$AP Ferg            Plain Jane     Still Striving   
                     Plain Jane     Still Striving   
A.C.O                De Madrugada     De Madrugada   
                     De Madrugada     De Madrugada   
...                                            ...   
sofia shizuko        chichiriviche   chichiriviche   
                     chichiriviche   chichiriviche   
                     chichiriviche   chichiriviche   
                     chichiriviche   chichiriviche   
Ñengo El Quetzal     Plakka Plakka             NaN   

                                                                     uri saved  
artist               track                                                      
A Tribe Called Quest Excursions                                      NaN   NaN  
A$AP Ferg            Plain Jane     spotify:track:4dVpf9jZjcORqGTLUaeYj9  True  
                     Plain Jane     spotify:track:4dVpf9jZjcORqGTLUaeYj9  True  
A.C.O                De Madrugada   spotify:track:280QqgbMrLeMs6B80mIPG2  True  
                     De Madrugada   spotify:track:280QqgbMrLeMs6B80mIPG2  True  
...                                                                  ...   ...  
sofia shizuko        chichiriviche  spotify:track:7MNtRrUXrsQwR10o3lvgDP  True  
                     chichiriviche  spotify:track:7MNtRrUXrsQwR10o3lvgDP  True  
                     chichiriviche  spotify:track:7MNtRrUXrsQwR10o3lvgDP  True  
                     chichiriviche  spotify:track:7MNtRrUXrsQwR10o3lvgDP  True  
Ñengo El Quetzal     Plakka Plakka                                   NaN   NaN  

[9392 rows x 5 columns]

## Before searching all of them, maybe we have them in my library

In [25]:
with open(mydata.joinpath("YourLibrary.json"), "r") as file:
    library = json.load(file)

In [26]:
library_tracks = pd.DataFrame(library['tracks']).clean_names()

In [27]:
library_tracks.shape

(1746, 4)

In [28]:
library_tracks.head(15)

,artist,album,track,uri
0,Nanpa Básico,Como Bolero,Como Bolero,spotify:track:7pOBTfz9EOi9Yb64cOI6kv
1,Geassassin,Right Now,Right Now,spotify:track:1TqEsVQvnvs2H9kg4HkqQG
2,Robot95,Bullets,Bullets,spotify:track:36i3GBIGaw0i1Gv6Kz6EPM
3,Neto Reyno,Nr Hits,Vida Callejera,spotify:track:0N7XU51NdMaXgdq8h6iweg
4,Tame Impala,The Slow Rush,Borderline,spotify:track:5hM5arv9KDbCHS0k9uqwjr
5,Hordatoj,Beat Tape (Volumen 1),C.u.e.,spotify:track:1VudAJHMGsEYjgI4ltjLKx
6,Akwid,KOMP 104.9 Radio Compa,Mi Afición,spotify:track:2E36sO4oJvuTmhCt1ImjNB
7,Café Tacvba,Avalancha de éxitos,Como te extraño mi amor,spotify:track:6hFHsQWB7HdVrSe7efRR82
8,Fntxy,Hvbitö,K Y,spotify:track:4iCgDMx1PEFrulIOgt3H3C
9,Robot95,Too Much Flow,Too Much Flow,spotify:track:2ZBlDOQlq043eg34LwW54c


In [29]:
tracks_uris = library_tracks.set_index(["artist", "album", "track"]).join(already_saved.set_index(["artist", "album", "track"]), rsuffix = "_saved", how = "outer").reset_index()
tracks_uris

,artist,album,track,uri,uri_saved,saved
0,2Pac,All Eyez On Me,Ambitionz Az A Ridah,spotify:track:20fBuVybkHgjF6vNhSMROD,NaN,NaN
1,2Pac,All Eyez On Me,Only God Can Judge Me (ft. Rappin' 4-Tay),spotify:track:2Z6I3youZAlfRvsyzJAGQA,NaN,NaN
2,50 Cent,Get Rich Or Die Tryin',Patiently Waiting,spotify:track:3ORfa5ilEthp2U0TRcv7kv,NaN,NaN
3,A$AP Ferg,Still Striving,Plain Jane,spotify:track:4dVpf9jZjcORqGTLUaeYj9,spotify:track:4dVpf9jZjcORqGTLUaeYj9,True
4,A.C.O,Buscándote,Buscándote,spotify:track:5Qh1noyTdOz7d3GNTwwqJP,NaN,NaN
...,...,...,...,...,...,...
2066,sofia shizuko,chichiriviche,chichiriviche,spotify:track:7MNtRrUXrsQwR10o3lvgDP,spotify:track:7MNtRrUXrsQwR10o3lvgDP,True
2067,Águila Sativa,Low Rider 64,Low Rider 64,spotify:track:7oCTo2ktHD7mX2knemZohF,NaN,NaN
2068,Ñengo El Quetzal,El Rikis,El Rikis,spotify:track:2VLpj3su9MbHqMWtxEqZhx,NaN,NaN
2069,Ñengo El Quetzal,Plakka Plakka,Plakka Plakka,spotify:track:1jzhpaWxMF1x4UtHrrERQW,NaN,NaN


In [30]:
tracks_uris['track_uri'] = tracks_uris['uri_saved'].combine_first(tracks_uris['uri'])
tracks_uris = tracks_uris.drop(columns=["uri", "uri_saved"]).rename(columns = {"track_uri": "uri"})
tracks_uris['saved'] = tracks_uris['saved'].fillna(False)
tracks_uris = tracks_uris.drop_duplicates()
tracks_uris

,artist,album,track,saved,uri
0,2Pac,All Eyez On Me,Ambitionz Az A Ridah,False,spotify:track:20fBuVybkHgjF6vNhSMROD
1,2Pac,All Eyez On Me,Only God Can Judge Me (ft. Rappin' 4-Tay),False,spotify:track:2Z6I3youZAlfRvsyzJAGQA
2,50 Cent,Get Rich Or Die Tryin',Patiently Waiting,False,spotify:track:3ORfa5ilEthp2U0TRcv7kv
3,A$AP Ferg,Still Striving,Plain Jane,True,spotify:track:4dVpf9jZjcORqGTLUaeYj9
4,A.C.O,Buscándote,Buscándote,False,spotify:track:5Qh1noyTdOz7d3GNTwwqJP
...,...,...,...,...,...
2066,sofia shizuko,chichiriviche,chichiriviche,True,spotify:track:7MNtRrUXrsQwR10o3lvgDP
2067,Águila Sativa,Low Rider 64,Low Rider 64,False,spotify:track:7oCTo2ktHD7mX2knemZohF
2068,Ñengo El Quetzal,El Rikis,El Rikis,False,spotify:track:2VLpj3su9MbHqMWtxEqZhx
2069,Ñengo El Quetzal,Plakka Plakka,Plakka Plakka,False,spotify:track:1jzhpaWxMF1x4UtHrrERQW


In [31]:
working_df = data.set_index(["artist", "track"]).join(tracks_uris.set_index(["artist", "track"]), how = 'left').reset_index()

In [32]:
working_df.stb.missing()

,missing,total,percent
album,906,10075,8.992556
saved,906,10075,8.992556
uri,906,10075,8.992556
artist,0,10075,0.000000
track,0,10075,0.000000
endtime,0,10075,0.000000
msplayed,0,10075,0.000000


In [33]:
working_df.duplicated(subset=['artist', 'album', 'track', 'endtime']).sum()

120

In [34]:
working_df[working_df.duplicated(subset=['artist', 'album', 'track', 'endtime'])]

,artist,track,endtime,msplayed,album,saved,uri
152,Akwid,Mi Afición,2022-01-13 16:38,270293,KOMP 104.9 Radio Compa,False,spotify:track:5uA6YOi9X0wlW9HDtDlQH7
252,Aleman,Hacia Arriba,2022-01-13 01:38,230341,Pase de Abordar,False,spotify:track:7GEROcDGgLAZk2GEd1ivgo
255,Aleman,Hacia Arriba,2022-03-05 20:19,230341,Pase de Abordar,False,spotify:track:7GEROcDGgLAZk2GEd1ivgo
258,Aleman,Hacia Arriba,2022-07-20 23:18,230341,Pase de Abordar,False,spotify:track:7GEROcDGgLAZk2GEd1ivgo
261,Aleman,Hacia Arriba,2022-08-12 22:42,230341,Pase de Abordar,False,spotify:track:7GEROcDGgLAZk2GEd1ivgo
...,...,...,...,...,...,...,...
8820,Santa Fe Klan,Socios,2022-11-08 17:02,243000,Socios,True,spotify:track:7d8TPVWQLeW5PKrAb76V3N
9991,Yoga Fire,Trap Talk,2022-02-10 22:36,210050,Wavy Super Latin,False,spotify:track:0cfGMJlyI9kLCRqF4yv319
9993,Yoga Fire,Trap Talk,2022-04-09 03:05,210050,Wavy Super Latin,False,spotify:track:0cfGMJlyI9kLCRqF4yv319
9995,Yoga Fire,Trap Talk,2022-05-19 19:32,105633,Wavy Super Latin,False,spotify:track:0cfGMJlyI9kLCRqF4yv319


In [35]:
working_df.shape

(10075, 7)

In [36]:
working_df = working_df.drop_duplicates()

In [37]:
working_df

,artist,track,endtime,msplayed,album,saved,uri
0,A Tribe Called Quest,Excursions,2022-01-28 21:59,41865,NaN,NaN,NaN
1,A$AP Ferg,Plain Jane,2022-03-27 02:20,173600,Still Striving,True,spotify:track:4dVpf9jZjcORqGTLUaeYj9
2,A$AP Ferg,Plain Jane,2022-11-29 01:31,173600,Still Striving,True,spotify:track:4dVpf9jZjcORqGTLUaeYj9
3,A.C.O,De Madrugada,2022-01-21 02:55,220757,De Madrugada,True,spotify:track:280QqgbMrLeMs6B80mIPG2
4,A.C.O,De Madrugada,2022-07-18 15:48,220757,De Madrugada,True,spotify:track:280QqgbMrLeMs6B80mIPG2
...,...,...,...,...,...,...,...
10070,sofia shizuko,chichiriviche,2022-07-15 00:49,149000,chichiriviche,True,spotify:track:7MNtRrUXrsQwR10o3lvgDP
10071,sofia shizuko,chichiriviche,2022-07-15 00:55,149000,chichiriviche,True,spotify:track:7MNtRrUXrsQwR10o3lvgDP
10072,sofia shizuko,chichiriviche,2022-07-16 18:59,69525,chichiriviche,True,spotify:track:7MNtRrUXrsQwR10o3lvgDP
10073,sofia shizuko,chichiriviche,2022-08-15 19:58,149000,chichiriviche,True,spotify:track:7MNtRrUXrsQwR10o3lvgDP


## Checkpoint

In [38]:
working_df.to_csv("processed/streaming_history_tracks.csv", encoding = 'utf-8', index = False)
podcast_data.to_csv("processed/streaming_history_pods.csv", encoding = 'utf-8', index = False)